In [1]:
# Importing libraries
import pandas as pd
from os import listdir

In [2]:
files = [file.split('.')[0] for file in listdir("../data")]

In [3]:
eval_expr = ', '.join(f'pd.read_csv(\'../data/{file}.csv\')' for file in files)

In [4]:
products, orders, departments, order_products_train, aisles, order_products_prior, sample_submission = eval(eval_expr)

In [5]:
# Shaping the data

prior_orders = pd.merge(orders, order_products_prior, on='order_id', how='inner')
prior_orders.head()


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [6]:
#Creating predictors

#Possible predictor categories:
#1. User predictors: How often does a user reorder? 
#2. Product Predictors: How often is a product reordered?
#3. User-Product Predictors: How often does a user buy a specific product?

In [7]:
total_orders_feature = prior_orders.groupby('user_id').agg(
    total_orders = pd.NamedAgg(column='order_number', aggfunc='max')
).reset_index()

In [8]:
user_reorder_ratio = prior_orders.groupby('user_id').agg(
    reorder_ratio = pd.NamedAgg(column='reordered', aggfunc='mean')
).reset_index()

In [9]:
# Joining the two user features on id 'user_id'
user_features = total_orders_feature.merge(user_reorder_ratio, on='user_id', how='inner')

In [31]:
# Product features

total_product_purchases = prior_orders.groupby('product_id').agg(
    total_product_purchases = pd.NamedAgg(column='order_id', aggfunc='count')
).reset_index()

In [22]:
product_reorder_ratio = prior_orders.groupby('product_id').agg(
    product_reorder_ratio = pd.NamedAgg(column='reordered', aggfunc='mean')
).reset_index()

In [33]:
product_reorder_ratio_filtered = product_reorder_ratio.loc[product_reorder_ratio['product_reorder_ratio'] > 0.5]

In [36]:
# Join product features
product_features = total_product_purchases.merge(product_reorder_ratio_filtered, on='product_id', how='left').fillna(0)

In [ ]:
# User-product predictors
total_product_buys = prior_orders.groupby(['user_id', 'product_id']).agg(
    total_product_buys = pd.NamedAgg(column='order_id', aggfunc='count')
).reset_index()